# Feature Extraction

We have extracted features from the frames using pre-trained VideoMAE model. We have saved these as .npy files which can be used for all the tasks.

In [2]:
import os, random, warnings
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import VideoMAEFeatureExtractor, VideoMAEModel
from tqdm import tqdm
from PIL import Image
import pandas as pd

class Config:
    FRAMES_ROOT  = "frames"
    METADATA_CSV = "FinalDataset.csv"
    MODEL_NAME   = "MCG-NJU/videomae-base"
    NUM_FRAMES   = 700
    CHUNK_SIZE   = 16
    OVERLAP      = 2
    FEATURES_DIR = "videomae_features_new"
    DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    CLIP_BATCH   = 4

random.seed(42); np.random.seed(42); torch.manual_seed(42)

class VideoDataset(Dataset):
    def __init__(self, df, fe, skip_existing=True):
        self.vids = df["video_id"].tolist()  # List of videos
        self.fe   = fe                       # Feature Extractor
        self.skip = skip_existing            # Skip existing videos
        os.makedirs(Config.FEATURES_DIR, exist_ok=True)

    def __len__(self): 
        return len(self.vids)

    def __getitem__(self, idx):
        vid = self.vids[idx]   # VideoID
        out_path = os.path.join(Config.FEATURES_DIR, f"{vid}_temporal.npy")  # Output path
        if self.skip and os.path.exists(out_path):   # Skip if the path already exists
            return None

        fdir   = os.path.join(Config.FRAMES_ROOT, vid)  # Video directory - Each video directory has 700 frames.
        fpaths = sorted([os.path.join(fdir, f)
                         for f in os.listdir(fdir)
                         if f.lower().endswith((".jpg"))])[:Config.NUM_FRAMES]  # Frame paths of the particular video

        frames = []
        for fp in fpaths:
            try:
                frames.append(Image.open(fp).convert("RGB"))# Open the image and convert it to RGB format - Convert method - Returns a converted copy of this image
            except Exception as e:
                warnings.warn(f"Skipped corrupt frame {fp}: {e}")  # Skips if it's not able to open or convert the image with a warning

        # Sliding window logic
        stride  = Config.CHUNK_SIZE - Config.OVERLAP  # Stride = 14
        clips   = []
        for i in range(0, len(frames), stride):  # Moves with a stride of 12, so the last two frames of the first one are added to the next clip
            clip = frames[i:i + Config.CHUNK_SIZE] # Get a overlapping clip of 16 frames
            if len(clip) < Config.CHUNK_SIZE:
                clip += [frames[-1]] * (Config.CHUNK_SIZE - len(clip))  # If the clip (mostly last) is less than 16 frames, pad it with a few frames from the previous chunk
            clips.append(clip) # Make a list of clips

        # Feature Extractor: Expects as input images/arrays
        # Output: [batch_size, num_frames, number_of_channels, height, width]
        tensors = []
        for clip in clips:
            pv = self.fe(clip, return_tensors="pt").pixel_values  # The feature extractor returns tensors of shape (1,16,3,224,224)
            tensors.append(pv)                      # Save tensors in a list
        pixel_values = torch.cat(tensors, dim=0)               # Concatenate all tensors along the batch dimension: (num_clips,16,3,224,224)
        return {"video_id": vid, "pixel_values": pixel_values, "n_frames": len(frames)}

# Take a sequence of token embeddings and convert them into frame-level embeddings
def tokens_to_frames(tok: torch.Tensor, n_frames=16):
    tok = tok[1:]                        # VideoMAE adds a [CLS] token at the beginning: drop CLS
    tokens_per_frame = tok.size(0) // n_frames  # Get the number of tokens per frame
    tok = tok[:tokens_per_frame * n_frames]  # Ensures the token count is exactly divisible by n_frames
    frames = tok.reshape(n_frames, tokens_per_frame, -1).mean(1) # Reshape and take mean across tokens for each frame
    return frames                        # (16,768) - 16 frame-level embeddings, each of size 768

def extract_temporal(skip_existing=True):
    df  = pd.read_csv(Config.METADATA_CSV)
    fe  = VideoMAEFeatureExtractor.from_pretrained(Config.MODEL_NAME) # Define the feature extractor
    model = VideoMAEModel.from_pretrained(Config.MODEL_NAME).to(Config.DEVICE).eval() # Pretrained videomae model

    ds = VideoDataset(df, fe, skip_existing)
    for sample in tqdm(ds, desc="videos"):
        if sample is None:
            continue

        vid    = sample["video_id"]
        inp    = sample["pixel_values"].to(Config.DEVICE)    # Input: (N,16,3,224,224)
        nfrm   = sample["n_frames"]
        stride = Config.CHUNK_SIZE - Config.OVERLAP
        feat_dim = 768

        clip_feats = []
        for i in range(0, inp.size(0), Config.CLIP_BATCH): # Process all clips (inp.size(0) is the number of clips per video)
            batch = inp[i:i+Config.CLIP_BATCH]               # keep (B,16,3,H,W)
            with torch.no_grad():                            # We're freezing the model weights
                out = model(batch).last_hidden_state         # Run batch through VideoMAE model: (B, seq+1, 768)
            for j in range(out.size(0)):
                clip_feats.append(tokens_to_frames(out[j], Config.CHUNK_SIZE).cpu()) # Convert tokens to frame-level features and save them in a list
        clip_feats = torch.stack(clip_feats)             # Combine features to form one tensor per video - (num_clips,16,768)

        blended = np.zeros((nfrm, feat_dim), dtype=np.float32) # Will store the final per-frame feature vectors.
        counts  = np.zeros(nfrm,      dtype=np.float32)   # Tracks how many times each frame is "touched" (due to overlap).

        w = np.ones(Config.CHUNK_SIZE, dtype=np.float32) # Will track weights used to blend clip features into final frame-wise features
        # If there is overlap, we need a smooth transition, so we give lower weights to the frames at the edges
        if Config.OVERLAP:
            w[:Config.OVERLAP]  = np.linspace(0, 1, Config.OVERLAP, endpoint=False) # First overlap - weights between 0 to 1
            w[-Config.OVERLAP:] = np.linspace(1, 0, Config.OVERLAP, endpoint=False) # Last overlap - weights between 1 to 0

        for k, clip in enumerate(clip_feats):  # (num_clips, 16, 768)
            start = k * stride   # first frame index
            end   = min(start + Config.CHUNK_SIZE, nfrm)  # last frame index
            w_use = w[:end - start]   # Extract relevant portion of the weight array
            blended[start:end] += clip[:end - start].numpy() * w_use[:, None] # Multiply the clips frame features with corresponding weights
            counts[start:end]  += w_use # Track the sum of weights for each frame so that we can normalize later

        blended[counts > 0] /= counts[counts > 0, None] # Weighted average

        np.save(os.path.join(Config.FEATURES_DIR, f"{vid}_temporal.npy"), blended) # Save the features

extract_temporal(skip_existing=True)
print("✓  Temporal features saved")

# Import libraries and set the seed

In [3]:
# ----------------------------- imports & config -----------------------------
import os
from pathlib import Path
import copy
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import optuna
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 5
N_EPOCHS = 40
PATIENCE = 10
N_TRIALS = 30
FEATURE_DIR = "videomae_features_new"
MODEL_DIR = Path("models_final") 
MODEL_DIR.mkdir(exist_ok=True)
RESULTS_DIR = Path("results_final") 
RESULTS_DIR.mkdir(exist_ok=True)
PIN = torch.cuda.is_available()
NUM_WORKERS = 0

def set_seed(seed: int = SEED):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

# Dataset Class

In [4]:
class TemporalFeatureDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, feature_dir: str, target_col: str = "log_view_count"):
        self.df = dataframe.reset_index(drop=True)
        self.feature_dir = Path(feature_dir)  # Base directory containing .npy feature files
        self.target_col = target_col

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]      # select the idx-th row
        vid = row["video_id"]        # fetch the video ID
        x = np.load(self.feature_dir / f"{vid}_temporal.npy")   # load features: expected shape [T, D]
        y = row[self.target_col]     # fetch the target for this sample
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32) # Convert to Pytorch tensors

def safe_pearsonr(preds: np.ndarray, targets: np.ndarray, eps: float = 1e-8) -> float:
    preds, targets = preds.squeeze(), targets.squeeze()   # Squeeze removes all dimensions of size 1
    cov = ((preds - preds.mean()) * (targets - targets.mean())).mean()  # Calculate the covariance
    return float(cov / ((preds.std() + eps) * (targets.std() + eps)))  # Calculate and return the pearson r - a small epsilon is added to avoid division by zero

def log_to_views(arr: np.ndarray) -> np.ndarray:
    return np.expm1(arr)        # Log view count is converted back to view -> Inverse of log1p

# Prediction Model - Transformer

In [5]:
class TransformerRegressor(nn.Module):
    def __init__(self, d_model, nhead, num_layers):
        super().__init__()
        # One encoder block (self-attention + FFN + residual + norm)
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead) # By default, batch_first=False (expects [T, B, D]).
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers) # Stack 'num_layers' encoder blocks
        self.regressor = nn.Linear(d_model, 1)  # Linear regression head mapping pooled [B, D] -> scalar

    def forward(self, x):  # x: (B, T, D) -> (5, 700, 758)
        x = x.permute(1, 0, 2)  # Permute as by default the transformer expects [T, B, D]
        x = self.transformer(x)  # (T, B, D)
        x = x.mean(dim=0)  # global average pooling over time
        return self.regressor(x).squeeze(-1) # Regress to a single scalar

# Training Loop

In [6]:
def train_one_epoch(model, loader, criterion, optimizer, clip_norm: float = 1.0):
    model.train()         # Put the model in training mode
    total_huber, preds, tgts = 0.0, [], []
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)  # Move the inputs and targets to the device (GPU)
        y_hat = model(x)     # Forward pass - Get the predictions
        loss = criterion(y_hat, y)    # Compute training loss (Huber)
        optimizer.zero_grad()       # Reset gradients 
        loss.backward()             # backpropagate to compute grads
        if clip_norm is not None:     # optionally clip gradient norm to stabilize training
            nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
        optimizer.step()             # update parameters
        total_huber += loss.item() * x.size(0)   # accumulate sum of loss over samples in this batch
        preds.append(y_hat.detach().cpu().numpy())   # Collect predictions on cpu for epoch metrics
        tgts.append(y.cpu().numpy())   # Collect targets on CPU
    preds = np.concatenate(preds)  # Concatenate per batch arrays into [N] arrays for whole epoch metrics
    tgts = np.concatenate(tgts)  
    mse = float(np.mean((preds - tgts) ** 2)) # Epoch MSE
    pc  = safe_pearsonr(preds, tgts)    # Epoch PC
    hub = total_huber / len(loader.dataset)   # Average huber over all samples in the epoch
    return hub, mse, pc

@torch.no_grad()     # Disable autograd for evaluation
def evaluate(model, loader):
    model.eval()    # Set the model in eval mode
    preds, tgts = [], []
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)   # Move to device
        preds.append(model(x).cpu().numpy())  # Forward pass -> Collect predictions
        tgts.append(y.cpu().numpy())      # Collect targets
    preds = np.concatenate(preds)     # [N] predictions
    tgts = np.concatenate(tgts)       # [N] targets
    mse = float(np.mean((preds - tgts) ** 2))   # MSE over full split
    pc  = safe_pearsonr(preds, tgts)      # PC over full split
    return mse, pc, preds, tgts

def train_and_evaluate(model, train_loader, val_loader, lr, weight_decay,
                       n_epochs=N_EPOCHS, patience=PATIENCE, verbose=True):
    crit = nn.HuberLoss(delta=1.0)         # Training criteria - Robust to outliers  
    opt  = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)   # Adam optimizer

    best_val = float("inf") 
    best_pc = None
    best_state = None; bad = 0

    # Epoch Loop
    for ep in range(1, n_epochs + 1):
        tr_hub, tr_mse, tr_pc = train_one_epoch(model, train_loader, crit, opt)  # one training epoch
        val_mse, val_pc, _, _ = evaluate(model, val_loader)    # evaluation on validation split
        if verbose:
            print(f"Epoch {ep:02d}: Train Huber={tr_hub:.4f}, Train MSE={tr_mse:.4f}, PC={tr_pc:.4f} | "
                  f"Val MSE={val_mse:.4f}, PC={val_pc:.4f}")
        if val_mse < best_val:   # If there is improvement in the validation MSE, save that as the best MSE and PC
            best_val, best_pc = float(val_mse), float(val_pc)  
            best_state = copy.deepcopy(model.state_dict())
            bad = 0
        else:
            bad += 1
            if bad >= patience:
                if verbose: print("Early stopping"); break

    model.load_state_dict(best_state)   # restore best checkpoint before returning
    return model, best_val, best_pc

# Optuna for Hyperparameter Tuning

In [7]:
# ------------------------------ optuna objective ----------------------------
def objective_builder(train_df, val_df, input_dim):
    def objective(trial):
        # loaders (trial-specific RNG for shuffling)
        print(f"\n[trial {trial.number}] build loaders...", flush=True)
        g = torch.Generator().manual_seed(SEED + trial.number)  # per-trial determenistic shuffle
        train_loader = DataLoader(
            TemporalFeatureDataset(train_df, FEATURE_DIR), # dataset for training
            batch_size=BATCH_SIZE, shuffle=True, generator=g, 
            num_workers=NUM_WORKERS, pin_memory=PIN       # pin mem if on CUDA for faster host->device copies
        )
        val_loader = DataLoader(
            TemporalFeatureDataset(val_df, FEATURE_DIR),
            batch_size=BATCH_SIZE, shuffle=False,    # No shuffle in case of validation
            num_workers=NUM_WORKERS, pin_memory=PIN
        )

        # hyperparams
        nhead   = trial.suggest_categorical("nhead", [4, 8])  # Suggests either 4 or 8
        num_layers      = trial.suggest_int("num_layers", 1, 4)   # Suggests any integer between 1 and 4
        lr              = trial.suggest_float("lr", 1e-4, 1e-2, log=True)   # Suggests any positive float between the range sampled log uniformly
        weight_decay    = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True) # Any positive float - also log uniform

        model = TransformerRegressor(d_model=768, nhead=nhead, num_layers=num_layers).to(DEVICE) # Create an instance of the model
        model, val_mse, val_pc = train_and_evaluate(model, train_loader, val_loader, lr, weight_decay, verbose=True) # Train and evaluate
        
        trial.set_user_attr("val_pc", float(val_pc)) # store Pearson on val
        trial.set_user_attr("state_dict", copy.deepcopy(model.state_dict())) # Stores a full copy of weights on every trial
        return float(val_mse) # Optuna will minimize this objective 
    return objective

In [8]:
# One function to run the full experiment
def run_split(name: str, train_ids: list, val_ids: list, input_dim: int = 768):
    print(f"\n=== Running split: {name} ===")
    df = pd.read_csv("FinalDataset.csv") # load target dataframe with log_view_count and video_id
    
    train_df = df[df.video_id.isin(train_ids)].reset_index(drop=True)  # Prepare the train DF
    val_df   = df[df.video_id.isin(val_ids)].reset_index(drop=True)    # Prepare the validation DF

    # run Optuna
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=SEED))  # TPESampler with a fixed seed for reproducible sampling order
    study.optimize(objective_builder(train_df, val_df, input_dim), n_trials=N_TRIALS)
    best = study.best_params    # Extract the trial's params and score
    print("Best hyper-parameters:", best)
    print("Best Val MSE (Optuna):", study.best_trial.value)

    # rebuild best model and load the exact best-epoch weights saved in the trial
    model = TransformerRegressor(d_model=768, nhead=best["nhead"], num_layers=best["num_layers"]).to(DEVICE)
    model.load_state_dict(study.best_trial.user_attrs["state_dict"])  # Restore the best state dict

    # eval loaders (no shuffle)
    train_loader = DataLoader(TemporalFeatureDataset(train_df, FEATURE_DIR),
                              batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN)
    val_loader   = DataLoader(TemporalFeatureDataset(val_df, FEATURE_DIR),
                              batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN)

    tr_mse, tr_pc, _, _ = evaluate(model, train_loader)
    vl_mse, vl_pc, vl_preds, vl_tgts = evaluate(model, val_loader)
    print(f"Reloaded eval — Train MSE={tr_mse:.4f}, PC={tr_pc:.4f} | Val MSE={vl_mse:.4f}, PC={vl_pc:.4f} "
          f"| ΔMSE={abs(vl_mse - study.best_trial.value):.6f}")   # Check that reloaded model gives the same MSE

    # save bundle + study
    bundle_path = MODEL_DIR / f"video_reg_{name}.pt"
    torch.save({
        "state_dict": study.best_trial.user_attrs["state_dict"],   # Exact best weights
        "best_params": best,                     # Optuna best parameters
        "val_mse": study.best_trial.value,   # Best val MSE
        "val_pc": study.best_trial.user_attrs["val_pc"],  # Best val PC
    }, bundle_path)

    with open(MODEL_DIR / f"video_reg_{name}.pkl", "wb") as f:
        pickle.dump(study, f)      # Save the entire study

    # Save the validation predictions for late-fusion
    out = pd.DataFrame({
        "video_id": val_df["video_id"],
        "y_true" : vl_tgts.astype(float),
        "y_pred"  : vl_preds.astype(float),
    })
    fname = RESULTS_DIR / f"video_reg_val_{name}.csv"
    out.to_csv(fname, index=False)
    print(f"Saved VAL preds → {fname}")

    return {
        "model": model, "train_df": train_df, "val_df": val_df,
        "vl_preds": vl_preds, "vl_tgts": vl_tgts, "vl_mse": vl_mse, "vl_pc": vl_pc,
        "bundle_path": bundle_path
    }

# Experiments

## Person-Independent Split

In [7]:
df = pd.read_csv('FinalDataset.csv')

train_ids_ind = set(os.listdir('Person-Independent_Split/train'))
val_ids_ind = set(os.listdir('Person-Independent_Split/val'))

res_ind = run_split("ind", train_ids_ind, val_ids_ind, input_dim=768)

[I 2025-08-17 13:22:54,914] A new study created in memory with name: no-name-8e426c5e-188e-4f76-ab28-8e0b8ceef538



=== Running split: ind ===

[trial 0] build loaders...


C:\ProgramData\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 01: Train Huber=3.0442, Train MSE=19.4923, PC=0.1382 | Val MSE=16.1234, PC=-0.0613
Epoch 02: Train Huber=2.5197, Train MSE=13.9490, PC=0.3691 | Val MSE=24.5051, PC=0.0154
Epoch 03: Train Huber=2.4539, Train MSE=15.2144, PC=0.3670 | Val MSE=17.1605, PC=0.0925
Epoch 04: Train Huber=2.3317, Train MSE=14.0142, PC=0.4078 | Val MSE=19.6433, PC=0.1440
Epoch 05: Train Huber=2.3208, Train MSE=13.8958, PC=0.4347 | Val MSE=23.6013, PC=-0.0262
Epoch 06: Train Huber=2.3411, Train MSE=14.2941, PC=0.4217 | Val MSE=18.8402, PC=0.0941
Epoch 07: Train Huber=2.2435, Train MSE=13.4543, PC=0.4481 | Val MSE=15.1642, PC=0.1950
Epoch 08: Train Huber=2.2803, Train MSE=13.2981, PC=0.4486 | Val MSE=20.0946, PC=0.0837
Epoch 09: Train Huber=2.2356, Train MSE=13.0736, PC=0.4523 | Val MSE=17.9258, PC=0.1207
Epoch 10: Train Huber=2.1923, Train MSE=12.8365, PC=0.4626 | Val MSE=21.5928, PC=0.0228
Epoch 11: Train Huber=2.2290, Train MSE=13.0291, PC=0.4563 | Val MSE=19.8727, PC=0.0958
Epoch 12: Train Huber=2.2213, 

[I 2025-08-17 13:24:40,272] Trial 0 finished with value: 15.164175033569336 and parameters: {'hidden_dim': 232, 'nhead': 4, 'num_layers': 3, 'lr': 0.0002051338263087451, 'weight_decay': 2.9375384576328313e-06}. Best is trial 0 with value: 15.164175033569336.


...

Epoch 13: Train Huber=3.0107, Train MSE=17.1503, PC=0.0214 | Val MSE=17.8903, PC=-0.0520
Early stopping

[trial 28] build loaders...
Epoch 01: Train Huber=3.4026, Train MSE=22.2245, PC=-0.0211 | Val MSE=20.5531, PC=0.0107
Epoch 02: Train Huber=3.0377, Train MSE=17.0082, PC=-0.0508 | Val MSE=28.1583, PC=0.0501
Epoch 03: Train Huber=3.0973, Train MSE=17.8484, PC=0.0472 | Val MSE=14.5627, PC=-0.0725
Epoch 04: Train Huber=3.0738, Train MSE=17.3741, PC=-0.0424 | Val MSE=15.8190, PC=0.0851
Epoch 05: Train Huber=3.0548, Train MSE=17.6489, PC=-0.0005 | Val MSE=29.5040, PC=-0.0213
Epoch 06: Train Huber=3.0797, Train MSE=18.3129, PC=0.0526 | Val MSE=17.8074, PC=-0.0428
Epoch 07: Train Huber=3.2949, Train MSE=21.1157, PC=-0.0617 | Val MSE=15.5608, PC=-0.0492
Epoch 08: Train Huber=2.9811, Train MSE=16.4921, PC=0.1156 | Val MSE=22.2835, PC=0.0225
Epoch 09: Train Huber=2.9483, Train MSE=16.5455, PC=0.0818 | Val MSE=13.9498, PC=0.0086
Epoch 10: Train Huber=2.9899, Train MSE=16.4488, PC=0.0291 | Val M

[I 2025-08-17 14:02:11,628] Trial 28 finished with value: 13.629014015197754 and parameters: {'hidden_dim': 263, 'nhead': 8, 'num_layers': 4, 'lr': 0.002851507546999549, 'weight_decay': 6.1768584114518685e-06}. Best is trial 14 with value: 13.354726791381836.


Epoch 29: Train Huber=3.0627, Train MSE=17.2630, PC=-0.1183 | Val MSE=19.3321, PC=0.0269
Early stopping

[trial 29] build loaders...
Epoch 01: Train Huber=3.0461, Train MSE=19.5714, PC=0.1678 | Val MSE=13.9921, PC=0.1163
Epoch 02: Train Huber=2.6120, Train MSE=15.3339, PC=0.3184 | Val MSE=18.7341, PC=0.0984
Epoch 03: Train Huber=2.3661, Train MSE=14.0815, PC=0.3952 | Val MSE=19.7116, PC=0.1010
Epoch 04: Train Huber=2.2605, Train MSE=13.7579, PC=0.4326 | Val MSE=21.6956, PC=0.1179
Epoch 05: Train Huber=2.4787, Train MSE=14.9964, PC=0.3603 | Val MSE=18.9520, PC=0.1985
Epoch 06: Train Huber=2.3573, Train MSE=14.3683, PC=0.3977 | Val MSE=25.6734, PC=-0.0344
Epoch 07: Train Huber=2.3485, Train MSE=13.4240, PC=0.4210 | Val MSE=16.8253, PC=0.1301
Epoch 08: Train Huber=2.2878, Train MSE=13.5647, PC=0.4217 | Val MSE=17.4668, PC=0.2113
Epoch 09: Train Huber=2.6627, Train MSE=15.9513, PC=0.2825 | Val MSE=24.4803, PC=-0.1127
Epoch 10: Train Huber=2.3887, Train MSE=13.8036, PC=0.3948 | Val MSE=17.3

[I 2025-08-17 14:02:59,671] Trial 29 finished with value: 13.992088317871094 and parameters: {'hidden_dim': 207, 'nhead': 4, 'num_layers': 2, 'lr': 0.0005114187209362978, 'weight_decay': 1.0027558934608288e-06}. Best is trial 14 with value: 13.354726791381836.


Epoch 11: Train Huber=2.2938, Train MSE=12.9832, PC=0.4552 | Val MSE=15.5455, PC=0.1387
Early stopping
Best hyper-parameters: {'hidden_dim': 380, 'nhead': 4, 'num_layers': 2, 'lr': 0.0009819347379144987, 'weight_decay': 3.491097047653223e-06}
Best Val MSE (Optuna): 13.354726791381836
Reloaded eval — Train MSE=15.5618, PC=0.3269 | Val MSE=13.3547, PC=0.1905 | ΔMSE=0.000000
Saved VAL preds → results_final\video_reg_val_ind.csv


In [9]:
import torch

blob = torch.load('models_final/video_reg_ind.pt', map_location=DEVICE)
best_params = blob["best_params"]

best_model = TransformerRegressor(
    d_model=768,
    nhead=best_params["nhead"],
    num_layers=best_params["num_layers"]).to(DEVICE)

best_model.load_state_dict(blob["state_dict"])
best_model.eval()

df = pd.read_csv('FinalDataset.csv')

train_ids_ind = set(os.listdir('Person-Independent_Split/train'))
val_ids_ind = set(os.listdir('Person-Independent_Split/val'))
train_df_ind = df[df.video_id.isin(train_ids_ind)].reset_index(drop=True)
val_df_ind   = df[df.video_id.isin(val_ids_ind)].reset_index(drop=True)

# Final evaluation metrics
g = torch.Generator().manual_seed(SEED)
train_loader = DataLoader(TemporalFeatureDataset(train_df_ind, 'videomae_features_new'), batch_size=BATCH_SIZE, shuffle=True, generator=g, 
                          num_workers=0, pin_memory=True)
val_loader = DataLoader(TemporalFeatureDataset(val_df_ind, 'videomae_features_new'),batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=0, pin_memory=True)
tr_mse, tr_pc, tr_preds, tr_tgts = evaluate(best_model, train_loader)
vl_mse, vl_pc, vl_preds, vl_tgts = evaluate(best_model, val_loader)
print(f"Best Val (Optuna): MSE={blob["val_mse"]:.4f}, PC={blob['val_pc']:.4f}")
print(f"Reloaded model eval: Val MSE={vl_mse:.4f}, PC={vl_pc:.4f}")

C:\Users\Mansi Jadhav\AppData\Local\Temp\ipykernel_31360\3011977617.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  blob = torch.load('models_final/video_reg_ind.pt', ma

Best Val (Optuna): MSE=13.3547, PC=0.1905
Reloaded model eval: Val MSE=13.3547, PC=0.1905


# Person-Dependent Split

In [24]:
df = pd.read_csv('FinalDataset.csv')

train_ids_dep = set(os.listdir('Person-Dependent_Split/train'))
val_ids_dep = set(os.listdir('Person-Dependent_Split/val'))

res_ind = run_split("dep", train_ids_dep, val_ids_dep, input_dim=768)

[I 2025-08-17 10:37:32,354] A new study created in memory with name: no-name-b64a9f5f-9674-4bca-88fd-399f0e21a148



=== Running split: dep ===

[trial 0] build loaders...


C:\ProgramData\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 01: Train Huber=3.0603, Train MSE=19.5646, PC=0.2249 | Val MSE=17.5592, PC=0.4534
Epoch 02: Train Huber=2.6461, Train MSE=15.5485, PC=0.4136 | Val MSE=18.2344, PC=0.4385
Epoch 03: Train Huber=2.4218, Train MSE=14.5399, PC=0.4530 | Val MSE=21.8907, PC=0.5580
Epoch 04: Train Huber=2.3868, Train MSE=14.1763, PC=0.4913 | Val MSE=18.4461, PC=0.4102
Epoch 05: Train Huber=2.4785, Train MSE=14.1192, PC=0.4708 | Val MSE=19.6633, PC=0.2445
Epoch 06: Train Huber=2.5270, Train MSE=15.1056, PC=0.4323 | Val MSE=16.6824, PC=0.4871
Epoch 07: Train Huber=2.3647, Train MSE=14.1475, PC=0.4836 | Val MSE=17.5346, PC=0.4226
Epoch 08: Train Huber=2.3848, Train MSE=13.9005, PC=0.4805 | Val MSE=18.4053, PC=0.3748
Epoch 09: Train Huber=2.4369, Train MSE=14.3565, PC=0.4601 | Val MSE=16.8220, PC=0.5139
Epoch 10: Train Huber=2.3026, Train MSE=13.5091, PC=0.5081 | Val MSE=16.4718, PC=0.5173
Epoch 11: Train Huber=2.2882, Train MSE=13.4011, PC=0.5208 | Val MSE=17.7211, PC=0.4849
Epoch 12: Train Huber=2.3194, Tr

[I 2025-08-17 10:41:01,620] Trial 0 finished with value: 14.763419151306152 and parameters: {'hidden_dim': 232, 'nhead': 4, 'num_layers': 3, 'lr': 0.0002051338263087451, 'weight_decay': 2.9375384576328313e-06}. Best is trial 0 with value: 14.763419151306152.


...

Epoch 17: Train Huber=2.1644, Train MSE=13.0492, PC=0.5451 | Val MSE=19.6766, PC=0.3578
Early stopping

[trial 28] build loaders...
Epoch 01: Train Huber=3.0167, Train MSE=19.7080, PC=0.2305 | Val MSE=24.1116, PC=0.4492
Epoch 02: Train Huber=2.5503, Train MSE=14.9596, PC=0.4256 | Val MSE=16.5580, PC=0.4668
Epoch 03: Train Huber=2.4392, Train MSE=14.1511, PC=0.4715 | Val MSE=17.5622, PC=0.5271
Epoch 04: Train Huber=2.4847, Train MSE=15.2876, PC=0.4448 | Val MSE=17.7925, PC=0.5366
Epoch 05: Train Huber=2.3567, Train MSE=14.1490, PC=0.4864 | Val MSE=14.8978, PC=0.5568
Epoch 06: Train Huber=2.2135, Train MSE=13.1633, PC=0.5337 | Val MSE=15.5033, PC=0.5621
Epoch 07: Train Huber=2.1227, Train MSE=12.4955, PC=0.5640 | Val MSE=17.0904, PC=0.5906
Epoch 08: Train Huber=2.0921, Train MSE=11.9419, PC=0.5828 | Val MSE=15.4197, PC=0.5813
Epoch 09: Train Huber=2.0747, Train MSE=12.5318, PC=0.5667 | Val MSE=14.9015, PC=0.5736
Epoch 10: Train Huber=1.9529, Train MSE=10.8438, PC=0.6304 | Val MSE=15.4725

[I 2025-08-17 11:21:00,486] Trial 28 finished with value: 13.710877418518066 and parameters: {'hidden_dim': 322, 'nhead': 4, 'num_layers': 1, 'lr': 0.00014914799080664363, 'weight_decay': 9.383844544152492e-06}. Best is trial 9 with value: 11.634990692138672.


Epoch 26: Train Huber=1.1316, Train MSE=5.1059, PC=0.8447 | Val MSE=15.6164, PC=0.5404
Early stopping

[trial 29] build loaders...
Epoch 01: Train Huber=3.0568, Train MSE=20.7987, PC=0.2217 | Val MSE=17.0595, PC=0.4649
Epoch 02: Train Huber=2.5239, Train MSE=15.3189, PC=0.4401 | Val MSE=16.6978, PC=0.4831
Epoch 03: Train Huber=2.5911, Train MSE=16.2991, PC=0.3825 | Val MSE=17.6554, PC=0.4163
Epoch 04: Train Huber=2.4101, Train MSE=14.2364, PC=0.4795 | Val MSE=18.1350, PC=0.5582
Epoch 05: Train Huber=2.3859, Train MSE=14.5163, PC=0.4702 | Val MSE=15.7348, PC=0.4957
Epoch 06: Train Huber=2.4069, Train MSE=14.5794, PC=0.4708 | Val MSE=15.8341, PC=0.5695
Epoch 07: Train Huber=2.4214, Train MSE=14.7618, PC=0.4661 | Val MSE=20.3454, PC=0.5047
Epoch 08: Train Huber=2.2792, Train MSE=13.5419, PC=0.5100 | Val MSE=17.7422, PC=0.5395
Epoch 09: Train Huber=2.3559, Train MSE=14.0017, PC=0.4927 | Val MSE=16.2429, PC=0.5358
Epoch 10: Train Huber=2.2528, Train MSE=13.0708, PC=0.5483 | Val MSE=17.8376,

[I 2025-08-17 11:21:53,215] Trial 29 finished with value: 13.948163032531738 and parameters: {'hidden_dim': 241, 'nhead': 4, 'num_layers': 1, 'lr': 0.0009652973366171388, 'weight_decay': 3.846546389413561e-06}. Best is trial 9 with value: 11.634990692138672.


Epoch 21: Train Huber=1.4185, Train MSE=6.7159, PC=0.7923 | Val MSE=22.3458, PC=0.3216
Early stopping
Best hyper-parameters: {'hidden_dim': 332, 'nhead': 4, 'num_layers': 1, 'lr': 0.00012315571723666037, 'weight_decay': 9.462175356461487e-06}
Best Val MSE (Optuna): 11.634990692138672
Reloaded eval — Train MSE=4.7953, PC=0.8565 | Val MSE=11.6350, PC=0.6951 | ΔMSE=0.000000
Saved VAL preds → results_final\video_reg_val_dep.csv


In [49]:
import torch

blob = torch.load('models_final/video_reg_dep.pt', map_location=DEVICE)
best_params = blob["best_params"]

best_model = TransformerRegressor(
    d_model=768,
    nhead=best_params["nhead"],
    num_layers=best_params["num_layers"]).to(DEVICE)

best_model.load_state_dict(blob["state_dict"])
best_model.eval()

df = pd.read_csv('FinalDataset.csv')

train_ids_dep = set(os.listdir('Person-Dependent_Split/train'))
val_ids_dep = set(os.listdir('Person-Dependent_Split/val'))
train_df_dep = df[df.video_id.isin(train_ids_dep)].reset_index(drop=True)
val_df_dep   = df[df.video_id.isin(val_ids_dep)].reset_index(drop=True)

# Final evaluation metrics
g = torch.Generator().manual_seed(SEED)
train_loader = DataLoader(TemporalFeatureDataset(train_df_dep, 'videomae_features_new'), batch_size=BATCH_SIZE, shuffle=True, generator=g, 
                          num_workers=0, pin_memory=True)
val_loader = DataLoader(TemporalFeatureDataset(val_df_dep, 'videomae_features_new'),batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=0, pin_memory=True)
tr_mse, tr_pc, tr_preds, tr_tgts = evaluate(best_model, train_loader)
vl_mse, vl_pc, vl_preds, vl_tgts = evaluate(best_model, val_loader)
print(f"Best Val (Optuna): MSE={blob["val_mse"]:.4f}, PC={blob['val_pc']:.4f}")
print(f"Reloaded model eval: Val MSE={vl_mse:.4f}, PC={vl_pc:.4f}")

C:\Users\Mansi Jadhav\AppData\Local\Temp\ipykernel_35428\4139045899.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  blob = torch.load('models_final/video_reg_dep.pt', ma

Best Val (Optuna): MSE=11.6350, PC=0.6951
Reloaded model eval: Val MSE=11.6350, PC=0.6951


# Test Predictions

In [1]:
import os
import json
import math
import copy
import numpy as np
import pandas as pd
from pathlib import Path

# Ensure reproducibility
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  # for deterministic CuBLAS paths

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
import pickle

FEATURE_DIR = "videomae_features_new"
CSV_PATH    = "FinalDataset.csv" 

MODEL_DIR   = Path("models_final")  
MODEL_DIR.mkdir(exist_ok=True)
RESULTS_DIR = Path("results_final") 
RESULTS_DIR.mkdir(exist_ok=True)

BATCH_SIZE_DEFAULT = 8
N_EPOCHS    = 40
PATIENCE    = 8
INPUT_DIM   = 768
NUM_WORKERS = 0
DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PIN         = torch.cuda.is_available()

LABEL_COL_MAP = {
    "ind": "view_range_enc_ind",
    "dep": "view_range_enc_dep",
}

# Set the seeds
def set_seed(seed=SEED):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

# Initialize each DataLoader worker with a different deterministic seed
def worker_init_fn(worker_id):
    s = SEED + worker_id
    np.random.seed(s)
    torch.manual_seed(s)

set_seed(SEED)

# Same as above
def safe_pearsonr(preds, targets, eps=1e-8):
    preds   = np.asarray(preds).squeeze()
    targets = np.asarray(targets).squeeze()
    cov = ((preds - preds.mean()) * (targets - targets.mean())).mean()
    return float(cov / ((preds.std() + eps) * (targets.std() + eps)))

def log_to_views(arr):
    return np.expm1(np.asarray(arr, dtype=float))

# Return set of file names without extension
def read_ids_from_dir(dir_path: str):
    assert os.path.isdir(dir_path), f"Missing directory: {dir_path}"
    return set(os.path.splitext(fn)[0] for fn in os.listdir(dir_path))

def get_dfs_for_split(split_tag: str):
    """
    Returns trainval_df and test_df for the split by reading id-lists from folders:
      Person-<Split>_Split/{train,val,test}
    Uses CSV with original encoded labels (no recomputation).
    """
    df = pd.read_csv(CSV_PATH)
    base = f"Person-{'Independent' if split_tag=='ind' else 'Dependent'}_Split"
    train_ids = read_ids_from_dir(os.path.join(base, "train"))
    val_ids   = read_ids_from_dir(os.path.join(base, "val"))
    test_ids  = read_ids_from_dir(os.path.join(base, "test"))

    trainval_ids = train_ids | val_ids
    trainval_df  = df[df.video_id.isin(trainval_ids)].reset_index(drop=True)  # train+val
    test_df      = df[df.video_id.isin(test_ids)].reset_index(drop=True)      # test

    need = {"video_id","channel_id","log_view_count","view_count",
            "view_range_enc_ind","view_range_enc_dep"}
    assert need.issubset(trainval_df.columns), f"Missing in trainval: {need - set(trainval_df.columns)}"
    assert need.issubset(test_df.columns), f"Missing in test: {need - set(test_df.columns)}"
    return trainval_df, test_df

def build_internal_val_from_labels(trainval_df: pd.DataFrame, split_tag: str,
                                   val_rel=0.15, seed=SEED):
    """
    Build ONE internal validation split from train+val using the PRECOMPUTED
    encoded labels (no new bins). PI: StratifiedGroupKFold (group=channel_id).
    PD: StratifiedKFold. We take the first fold to approximate ~val_rel.
    """
    label_col = LABEL_COL_MAP[split_tag]
    assert label_col in trainval_df.columns, f"Missing {label_col}."
    assert "channel_id" in trainval_df.columns, "Missing channel_id."

    y = pd.Series(trainval_df[label_col]).astype(str)
    if not y.str.fullmatch(r"\d+").all():
        y = pd.Series(pd.Categorical(y).codes)
    else:
        y = y.astype(int)

    n_splits = max(2, int(round(1.0 / val_rel)))  # e.g., 0.15 -> ~7
    idx = np.arange(len(trainval_df))

    if split_tag == "ind":
        groups = trainval_df["channel_id"].values
        cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=seed)   # no channel overlap for ind
        tr_idx, va_idx = next(cv.split(idx, y, groups=groups)) 
    else:
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)    # channel overlap
        tr_idx, va_idx = next(cv.split(idx.reshape(-1,1), y))

    return tr_idx, va_idx

def load_best_reg_bundle(split_tag: str):
    """
    Loads:
      - state_dict (if you saved one) from models*/video_reg_{split}.pt OR video_trans_reg_{split}.pt
      - best_params from accompanying pkl (optional)
    """
    pt_path = MODEL_DIR / f"video_reg_{split_tag}.pt"

    state_obj = torch.load(pt_path, map_location=DEVICE)
    if isinstance(state_obj, dict) and "state_dict" in state_obj:
        state_dict = state_obj["state_dict"]
        best_params_from_pt = state_obj.get("best_params", None)
    else:
        state_dict = state_obj
        best_params_from_pt = None

    pkl_path = MODEL_DIR / f"video_reg_{split_tag}.pkl"
    
    best_params = None
    if pkl_path.exists():
        with open(pkl_path, "rb") as f:
            bundle = pickle.load(f)
    best_params = bundle.get("best_params") or bundle.get("params") or None

    if best_params is None:
        best_params = best_params_from_pt

    return state_dict, best_params

# ======================= Runner (train+val -> test) ===========
def train_on_trainval_then_test_reg(split_tag: str):
    """
    Regression pipeline:
    - Load best params/state (state used only as init; params drive the model)
    - Create one internal val from train+val via PRECOMPUTED labels (no new bins)
    - Train with Huber, early stop on Val MSE (log space)
    - Evaluate on TEST: MSE(log), Pearson r, and MSE(views)
    - Save predictions CSV (video_id, y_true, y_pred) + save final train+val-tuned model
    """
    set_seed(SEED)
    print(f"\n===== [{split_tag}] REG: Train+Val -> Test (labels from CSV) =====")

    # Dataframes
    trainval_df, test_df = get_dfs_for_split(split_tag)
    print(f"train+val: {len(trainval_df)} | test: {len(test_df)}")

    # Load best bundle (state dict + hyperparams)
    state_dict, best_params = load_best_reg_bundle(split_tag)
    batch_size = int(best_params.get("batch_size", BATCH_SIZE_DEFAULT))

    # Build internal split for early stopping
    tr_idx, va_idx = build_internal_val_from_labels(trainval_df, split_tag=split_tag, val_rel=0.15, seed=SEED)
    tr_in = trainval_df.iloc[tr_idx].reset_index(drop=True)
    va_in = trainval_df.iloc[va_idx].reset_index(drop=True)
    print(f"internal train: {len(tr_in)} | internal val: {len(va_in)}")

    # Model
    model = TransformerRegressor(
        d_model=int(best_params["d_model"]),
        nhead=int(best_params["nhead"]),
        num_layers=int(best_params["num_layers"])
    ).to(DEVICE)

    # Loaders
    g = torch.Generator().manual_seed(SEED)
    tr_loader = DataLoader(
        TemporalFeatureDataset(tr_in, feature_dir=FEATURE_DIR, target_col="log_view_count"),
        batch_size=batch_size, shuffle=True, generator=g,
        num_workers=NUM_WORKERS, pin_memory=PIN,
        worker_init_fn=worker_init_fn if NUM_WORKERS>0 else None
    )
    va_loader = DataLoader(
        TemporalFeatureDataset(va_in, feature_dir=FEATURE_DIR, target_col="log_view_count"),
        batch_size=batch_size, shuffle=False,
        num_workers=NUM_WORKERS, pin_memory=PIN,
        worker_init_fn=worker_init_fn if NUM_WORKERS>0 else None
    )
    te_loader = DataLoader(
        TemporalFeatureDataset(test_df, feature_dir=FEATURE_DIR, target_col="log_view_count"),
        batch_size=batch_size, shuffle=False,
        num_workers=NUM_WORKERS, pin_memory=PIN,
        worker_init_fn=worker_init_fn if NUM_WORKERS>0 else None
    )

    # Train with early stopping on val MSE (log)
    criterion = nn.HuberLoss(delta=1.0)   # Huber loss
    optimizer = optim.Adam(model.parameters(), lr=float(best_params["lr"]), weight_decay=float(best_params["weight_decay"])) # Get the best params

    best_val, best_state, best_pc, best_epoch, bad = float("inf"), None, None, None, 0
    for ep in range(1, N_EPOCHS + 1):
        tr_hub, tr_mse, tr_pc = train_one_epoch(model, tr_loader, criterion, optimizer)  # Train the model
        va_mse, va_pc, _, _   = evaluate(model, va_loader)   # Evaluate
        print(f"Epoch {ep:02d} | Train Huber={tr_hub:.4f} MSE={tr_mse:.4f} PC={tr_pc:.4f} || "
              f"Val MSE={va_mse:.4f} PC={va_pc:.4f}")
        if va_mse < best_val:
            best_val, best_pc = float(va_mse), float(va_pc)
            best_state = copy.deepcopy(model.state_dict())
            best_epoch = ep
            bad = 0
        else:
            bad += 1
            if bad >= PATIENCE:
                print("Early stopping.")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    print(f"[{split_tag}] Best epoch: {best_epoch} | Best Val MSE={best_val:.4f} | Val PC={best_pc:.4f}")

    # Test evaluation
    te_mse, te_pc, te_preds, te_tgts = evaluate(model, te_loader)   # Evaluate on test
    te_views_mse = float(np.mean((log_to_views(te_preds) - log_to_views(te_tgts))**2))  # Get the raw views

    print(f"[{split_tag}] TEST — MSE(log)={te_mse:.4f} | PC={te_pc:.4f} | MSE(views)={te_views_mse:.2f}")

    # Save predictions CSV (slim)
    out = test_df[["video_id"]].copy()
    out["y_true"] = test_df["log_view_count"].to_numpy()
    out["y_pred"] = np.asarray(te_preds).reshape(-1)
    save_csv = RESULTS_DIR / f"video_reg_test_{split_tag}.csv"
    out.to_csv(save_csv, index=False)
    print(f"[{split_tag}] Saved test predictions to: {save_csv}")

    # Save the train+val-tuned model bundle
    final_bundle = {
        "state_dict": model.state_dict(),
        "best_params": best_params,
        "internal_val_mse": float(best_val),
        "internal_val_pc":  float(best_pc),
        "seed": SEED,
    }
    final_path = MODEL_DIR / f"video_reg_final_{split_tag}.pt"
    torch.save(final_bundle, final_path)
    print(f"[{split_tag}] Saved final model bundle to: {final_path}")


    return {
        "model": model,
        "test_mse_log": te_mse,
        "test_pc": te_pc,
        "test_mse_views": te_views_mse,
        "preds_path": save_csv,
        "best_epoch": best_epoch
    }

# ======================= Run both splits =====================
if __name__ == "__main__":
    res_ind = train_on_trainval_then_test_reg("ind")
    res_dep = train_on_trainval_then_test_reg("dep")


===== [ind] REG: Train+Val -> Test (labels from CSV) =====
train+val: 350 | test: 150
internal train: 295 | internal val: 55


C:\Users\Mansi Jadhav\AppData\Local\Temp\ipykernel_32448\1127137794.py:251: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_obj = torch.load(pt_path, map_location=DEVICE

[ind] Loaded init weights (strict=False).
Epoch 01 | Train Huber=3.1840 MSE=19.0002 PC=0.0163 || Val MSE=14.4978 PC=0.5847
Epoch 02 | Train Huber=2.7116 MSE=15.4706 PC=0.2398 || Val MSE=9.5465 PC=0.6348
Epoch 03 | Train Huber=2.3861 MSE=14.3495 PC=0.3510 || Val MSE=9.4672 PC=0.6280
Epoch 04 | Train Huber=2.3966 MSE=14.5108 PC=0.3724 || Val MSE=13.2801 PC=0.5669
Epoch 05 | Train Huber=2.3484 MSE=14.3251 PC=0.3838 || Val MSE=9.2055 PC=0.6272
Epoch 06 | Train Huber=2.1744 MSE=13.6041 PC=0.4307 || Val MSE=12.4418 PC=0.5989
Epoch 07 | Train Huber=2.1434 MSE=13.0432 PC=0.4721 || Val MSE=10.3460 PC=0.5998
Epoch 08 | Train Huber=2.0468 MSE=12.4753 PC=0.5089 || Val MSE=12.5429 PC=0.5426
Epoch 09 | Train Huber=2.0901 MSE=11.5850 PC=0.5304 || Val MSE=10.5114 PC=0.5850
Epoch 10 | Train Huber=2.0117 MSE=11.8975 PC=0.5578 || Val MSE=18.8625 PC=0.1444
Epoch 11 | Train Huber=1.9453 MSE=11.1193 PC=0.5646 || Val MSE=9.9445 PC=0.5935
Epoch 12 | Train Huber=2.0235 MSE=12.0965 PC=0.5234 || Val MSE=13.0835 

C:\Users\Mansi Jadhav\AppData\Local\Temp\ipykernel_32448\1127137794.py:251: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_obj = torch.load(pt_path, map_location=DEVICE

Epoch 01 | Train Huber=4.0380 MSE=31.6314 PC=-0.0489 || Val MSE=16.7622 PC=0.4311
Epoch 02 | Train Huber=3.0549 MSE=17.9320 PC=0.3453 || Val MSE=15.5354 PC=0.3350
Epoch 03 | Train Huber=2.8352 MSE=16.2316 PC=0.4239 || Val MSE=14.5150 PC=0.4098
Epoch 04 | Train Huber=2.6522 MSE=14.9041 PC=0.4697 || Val MSE=14.4166 PC=0.4126
Epoch 05 | Train Huber=2.5648 MSE=14.7092 PC=0.4526 || Val MSE=14.0705 PC=0.4162
Epoch 06 | Train Huber=2.4978 MSE=14.0182 PC=0.4968 || Val MSE=14.2284 PC=0.3976
Epoch 07 | Train Huber=2.3367 MSE=13.1949 PC=0.5280 || Val MSE=15.4379 PC=0.3607
Epoch 08 | Train Huber=2.2855 MSE=12.8853 PC=0.5479 || Val MSE=15.5132 PC=0.3993
Epoch 09 | Train Huber=2.2530 MSE=12.8772 PC=0.5525 || Val MSE=16.2282 PC=0.3484
Epoch 10 | Train Huber=2.1904 MSE=12.4974 PC=0.5678 || Val MSE=16.9641 PC=0.3223
Epoch 11 | Train Huber=2.0198 MSE=11.3784 PC=0.6176 || Val MSE=17.1819 PC=0.3213
Epoch 12 | Train Huber=1.9836 MSE=11.4122 PC=0.6191 || Val MSE=19.9119 PC=0.3393
Epoch 13 | Train Huber=2.06